In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

# ---------------- COLETA DE DADOS ---------------- #

# URLs oficiais dos programas sociais
urls = {
    "Bolsa Família": "https://www.gov.br/mds/pt-br/acoes-e-programas/bolsa-familia",
    "BPC": "https://www.gov.br/mds/pt-br/acoes-e-programas/suas/beneficios-assistenciais/beneficio-assistencial-ao-idoso-e-a-pessoa-com-deficiencia-bpc",

}

# Função para extrair texto das páginas
def extrair_texto(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        return soup.get_text(separator=" ").lower()
    else:
        print(f"Erro ao acessar {url}: {response.status_code}")
        return ""

# Função para extrair critérios automaticamente
def extrair_criterios(programa, texto):
    criterios = {
        "renda": {},
        "idade": {},
        "familia": {},
        "condicionalidades": [],
        "restricoes": {}
    }

    if programa == "Bolsa Família":
        renda = re.search(r"r\$\s*([0-9]+)\s*,?\.?([0-9]*)", texto)
        if renda:
            criterios["renda"]["tipo"] = "renda_per_capita"
            criterios["renda"]["limite"] = float(renda.group(1))
            criterios["renda"]["descricao"] = "Famílias com renda mensal per capita de até R$ {:.2f}.".format(float(renda.group(1)))
        
        criterios["familia"]["cadastro_unico"] = "cadastro único" in texto
        criterios["familia"]["descricao"] = "Famílias inscritas no Cadastro Único, com crianças, gestantes ou adolescentes."

        if "frequência escolar" in texto:
            criterios["condicionalidades"].append("Frequência escolar mínima para crianças e adolescentes")
        if "vacinação" in texto:
            criterios["condicionalidades"].append("Vacinação em dia")
        if "pré-natal" in texto:
            criterios["condicionalidades"].append("Acompanhamento pré-natal para gestantes")

    elif programa == "BPC":
        if "65" in texto:
            criterios["idade"]["minima"] = 65
            criterios["idade"]["descricao"] = "Destinado a idosos com 65 anos ou mais e pessoas com deficiência."
        if "1/4" in texto or "um quarto" in texto:
            criterios["renda"]["tipo"] = "renda_per_capita"
            criterios["renda"]["limite"] = "≤ 1/4 do salário mínimo"
            criterios["renda"]["descricao"] = "Famílias com renda mensal per capita de até 1/4 do salário mínimo."
        
        criterios["familia"]["cadastro_unico"] = "cadastro único" in texto
        criterios["familia"]["descricao"] = "Necessário estar inscrito no Cadastro Único e comprovar situação socioeconômica."

        if "aposentadoria" in texto or "pensão" in texto:
            criterios["restricoes"]["descricao"] = "Não pode acumular com aposentadoria ou pensão."

    return criterios


# ---------------- GERAÇÃO AUTOMÁTICA DO JSON ROTULADO ---------------- #

todos_criterios_rotulados = {"programas_sociais": []}

for programa, url in urls.items():
    texto = extrair_texto(url)
    criterios = extrair_criterios(programa, texto)

    # Geração automática do rótulo principal (categoria)
    if programa == "Bolsa Família":
        categoria = "Baixa Renda"
    elif programa == "BPC":
        categoria = "Terceira Idade"
    else:
        categoria = "Outros"

    todos_criterios_rotulados["programas_sociais"].append({
        "programa": programa,
        "rotulos": criterios,
        "categoria_gerada": categoria
    })

# Salvando o JSON rotulado
with open("criterios_rotulados.json", "w", encoding="utf-8") as f:
    json.dump(todos_criterios_rotulados, f, indent=2, ensure_ascii=False)

print("Arquivo criterios_rotulados.json gerado com sucesso!")
print("Saída gerada (JSON rotulado):\n")
print(json.dumps(todos_criterios_rotulados, indent=2, ensure_ascii=False))

Arquivo criterios_rotulados.json gerado com sucesso!
Saída gerada (JSON rotulado):

{
  "programas_sociais": [
    {
      "programa": "Bolsa Família",
      "rotulos": {
        "renda": {
          "tipo": "renda_per_capita",
          "limite": 218.0,
          "descricao": "Famílias com renda mensal per capita de até R$ 218.00."
        },
        "idade": {},
        "familia": {
          "cadastro_unico": true,
          "descricao": "Famílias inscritas no Cadastro Único, com crianças, gestantes ou adolescentes."
        },
        "condicionalidades": [],
        "restricoes": {}
      },
      "categoria_gerada": "Baixa Renda"
    },
    {
      "programa": "BPC",
      "rotulos": {
        "renda": {
          "tipo": "renda_per_capita",
          "limite": "≤ 1/4 do salário mínimo",
          "descricao": "Famílias com renda mensal per capita de até 1/4 do salário mínimo."
        },
        "idade": {
          "minima": 65,
          "descricao": "Destinado a idosos com 65